<a href="https://colab.research.google.com/github/RyannDaGreat/Practice/blob/master/tensorboard_nn_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Ryan Burgert Simple Tensorflow Experiments**

In [1]:
#Install my libraries
!pip install rp ipyplot

     |████████████████████████████████| 2.3MB 5.2MB/s 
  Created wheel for rp: filename=rp-0.1.498-cp36-none-any.whl size=1024535 sha256=10a005b9a33e06cfe0f3c241b157fc59032d4c1661fbabbb7590628811285797
  Stored in directory: /root/.cache/pip/wheels/d1/84/13/7bbb3a92b393e34d489da4ce7e5e889b9db84129a929d1cc7a
  Created wheel for doge: filename=doge-3.5.0-cp36-none-any.whl size=16562 sha256=e0e95ec6140938f7562a463a5cce0e200aef6e426dbbaabdeac146cfa51714d7
  Stored in directory: /root/.cache/pip/wheels/ba/de/0e/d150087c243f1c5c23d997f7852d99c6faf9f64bac7d23c09b
Successfully built rp doge


In [ ]:
#Get tensorboard ready
%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:
import rp
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf
import tensorflow.keras as keras
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train = np.expand_dims(x_train / 255.0, -1)
x_test  = np.expand_dims(x_test  / 255.0, -1)

In [ ]:
#Remove all old tensorboard logs
!rm -rf logs

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=50,kernel_size=3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation=tf.keras.activations.relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Softmax(),
])

loss = tf.keras.losses.SparseCategoricalCrossentropy()

metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]

optimizer = tf.keras.optimizers.Adam()

model.compile(optimizer, loss, metrics)

log_title='Conv'
log_path=rp.path_join('logs',log_title+"___"+str(rp.get_current_date()).replace(' ','_'))

rp.fansi_print("Log path: "+repr(log_path),'green','bold')

tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_path)

model.fit(x_train, y_train, epochs=10, batch_size=64, validation_split=.25, callbacks=[tensorboard])

rp.fansi_print('Evaluating test data:','green','bold')

model.evaluate(x_test,y_test)

In [ ]:
## Visualization of results

bins = [[] for _ in range(10)]
x_images=x_test[:800]
x_labels=model(x_images).numpy().squeeze().argmax(1)
x_images=x_images.squeeze()
for x_label,x_image in zip(x_labels,x_images):
    bins[x_label].append(x_image)
for _ in range(10):
    bins[_]=rp.horizontally_concatenated_images(rp.cv_text_to_image(str(_),scale=1),*bins[_])
visualization=rp.vertically_concatenated_images(bins)

rp.display_image(visualization)